Лабораторная работа №9


In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.linalg import sqrtm



# 1. Загрузите данные ex9_movies.mat из файла.

mat = loadmat('ex9_movies.mat')

Y = pd.DataFrame(mat["Y"])
R = pd.DataFrame(mat["R"])

In [2]:
# 2. Выберите число признаков фильмов (n) для реализации алгоритма коллаборативной фильтрации.

n = 1000

In [3]:
# 3. Реализуйте функцию стоимости для алгоритма.

def hypothesis(data: np.ndarray, thetas: np.ndarray):
    return data @ thetas.T

def cost(l: float, data: np.ndarray, y: np.ndarray, r: np.ndarray, thetas: np.ndarray):
    return (((hypothesis(data, thetas) - y) * r)**2).sum() + l * (thetas**2).sum() / 2


In [4]:
# 4. Реализуйте функцию вычисления градиентов.
# 5. При реализации используйте векторизацию для ускорения процесса обучения.

def gradient_descent(alpha: float, thetas: np.ndarray, data: np.ndarray, y: np.ndarray, r: np.ndarray, iterations=10**3):
    for i in range(iterations):
        data = data - alpha * (((hypothesis(data, thetas) - y) * r) @ thetas)
        thetas = thetas - alpha * (((hypothesis(data, thetas) - y) * r).T @ data)
    return thetas, data



In [5]:
# 6. Добавьте L2-регуляризацию в модель.

def gradient_descent_with_lambda(alpha: float, l: float, thetas: np.ndarray, data: np.ndarray, y: np.ndarray, r: np.ndarray, iterations=20**3):
    for i in range(iterations):
        data = data - alpha * (((hypothesis(data, thetas) - y) * r) @ thetas + l * data)
        thetas = thetas - alpha * (((hypothesis(data, thetas) - y) * r).T @ data + l * thetas)
    return thetas, data

In [6]:
# 7. Обучите модель с помощью градиентного спуска или других методов оптимизации.

start_data = np.random.uniform(0, 0.001, (len(Y), n))
start_thetas = np.random.uniform(0, 0.001, (len(Y.T), n))

mean = (Y.sum(axis=1) / R.sum(axis=1)).to_numpy().reshape(len(Y), 1)
Y_normalized = ((Y.to_numpy() - mean) * R).to_numpy()

l = 0.01
thetas, data = gradient_descent_with_lambda(0.001, l, start_thetas, start_data, Y_normalized, R.to_numpy())

print("Потери:", cost(l, data, Y_normalized, R.to_numpy(), thetas))

Потери: 29.40806469506791


In [7]:
# 8. Добавьте несколько оценок фильмов от себя. Файл movie_ids.txt содержит индексы каждого из фильмов.

def add_rating(films_rating: dict, total_number: int):
    custom_Y = np.zeros(total_number)
    custom_R = np.zeros(total_number)
    for key, val in films_rating.items():
        custom_R[key] = 1
        custom_Y[key] = val
    return custom_Y, custom_R

my_ratings = {1: 3, 29: 3, 69: 5, 71: 4, 82: 4, 94: 3, 95: 5, 96: 5, 231: 4, 254: 5, 252: 4, 541: 5, 665: 3, 755: 5, 810: 4, 820: 3, 50: 5}

custom_Y, custom_R = add_rating(my_ratings, len(Y))
R_with_custom = np.append(R, custom_R.reshape(-1, 1), 1)
Y_with_custom = np.append(Y, custom_Y.reshape(-1, 1), 1)
mean = (Y_with_custom.sum(axis=1) / R_with_custom.sum(axis=1)).reshape(len(Y_with_custom), 1)
Y_normalized = ((Y_with_custom - mean) * R_with_custom)

start_data = np.random.uniform(0, 0.001, (len(Y_with_custom), n))
start_thetas = np.random.uniform(0, 0.001, (len(Y_with_custom.T), n))

thetas, data = gradient_descent_with_lambda(0.01, l, start_thetas, start_data, Y_normalized, R_with_custom)

In [11]:
# 9. Сделайте рекомендации для себя. Совпали ли они с реальностью?

def prediction(data, thetas, index, mean):
    return (data[index] @ thetas[-1] + mean[index]).round()

require_recomendation = [890, 195, 1274]

recommendations = {}
for i in require_recomendation:
    recommendations[i] = prediction(data, thetas, i - 1, mean)

recommendations

{890: array([2.]), 195: array([4.]), 1274: array([2.])}

In [27]:
# 10. Также обучите модель с помощью сингулярного разложения матриц. Отличаются ли полученные результаты?

U, e, V = np.linalg.svd(data)
e = sqrtm(np.diag(e)[:n, :n])

matrix = (U[:, :n] @ e) @ (e @ V[:n, :])
prediction_matrix = matrix[:, -1].reshape(len(mean), 1) + mean

recommendations = {}
for i in require_recomendation:
    recommendations[i] = prediction_matrix[i - 1].round()

recommendations

{890: array([2.]), 195: array([4.]), 1274: array([2.])}